In [1]:
from sqlalchemy.orm import scoped_session, sessionmaker
from flask import Blueprint, Flask, session, request, jsonify
from decipher.framework.schema import engine, Problem
from decipher.framework.indexer import preprocess_text

session = scoped_session(sessionmaker(bind=engine))

In [2]:
problems = session.query(Problem).all()

terms = ['directed', 'undirected', 'vertices', 'edges']

term_ind = {}
temp_ind = 0 
for term in terms:
    term_ind[term] = temp_ind
    temp_ind+=1
    
doc_vector = [[0]*len(terms) for i in range(len(problems))]
doc_vector_ind = {}

temp_ind = 0
for problem in problems:
    doc_vector_ind[temp_ind] = problem.problem_id
    statement_p = preprocess_text(problem.statement)
    for i in statement_p[0]:
        if i in terms:
            doc_vector[temp_ind][term_ind[i]]+=1
    temp_ind+=1
#     print(statement_p)

In [3]:
problem_vectors = []
old_doc_vector_ind = doc_vector_ind.copy()
doc_vector_ind = {}
temp_ind = 0
for i in range(len(doc_vector)):
    if max(doc_vector[i])+min(doc_vector[i])!=0:
        doc_vector_ind[old_doc_vector_ind[i]] = temp_ind
        problem_vectors.append(tuple(doc_vector[i]))
        temp_ind+=1

In [4]:
import itertools
import collections
import numpy as np

class Solver:

    '''
    Solver class.
    '''

    def __init__(self, Y, M, epsilon, distance):
        '''
        Parameters
        ----------
            Y : list<vector>
                Finite set of vectors
            M : int
                Positive integer lesser than the size of Y
            epsilon : float
                Relative error
            distance : callable
                Distance metric
        '''
        self.Y = Y
        if isinstance(Y[0], int):
            self.q = 1
        else:
            self.q = len(Y[0])
        self.M = M
        self.epsilon = epsilon
        self.distance = distance

    def solve(self):        
        min_obj_fun_val = 1e10
        opt_subset = []
        n = len(self.Y)
        for y in self.Y:
            print (str(n) + ' objects remaining')
            n-=1
            
            ZMyY = self.computeZMvY(y) # M elements of Y closest to y

            rMyY = ZMyY[-1][-1] # maximal distance between y and elements in ZMyY
            
            h = self.epsilon*1.0 / (self.q*self.M)**0.5 * rMyY

            H = self.M**0.5 * rMyY

            if rMyY==0:
                return ZMyY, rMyY

            ByhH = self.generateByhH(y,h,H)
            
            nb = len(ByhH)
            
            for b in ByhH:
                
#                 print (str(nb) + ' subobjects remaining')
                nb-=1
                
                ZMbY = self.computeZMvY(b)

                subset = [i[0] for i in ZMbY]

                obj_fun_val = self.computeObj(subset)

                if obj_fun_val < min_obj_fun_val:
                    min_obj_fun_val = obj_fun_val
                    opt_subset = subset
        return opt_subset, min_obj_fun_val

    def generateByhH(self, y, h, H):
        if self.q==1:
            return np.hstack((
                np.arange(y, -1*H, h), np.arange(y, H, h)
            ))
        arr = [[] for i in range(self.q)]
        for i in  range(self.q):
            arr[i] = np.hstack((
                np.arange(y[i], -1*H, -1*h), np.arange(y[i], H, h)
            ))
        out = []
        n = self.q
        indices = [0 for i in range(n)]
        
        print ('generating ByhH. number of elements in each dimension: ', end = ' ')
        for i in range(len(arr)):
            print (len(arr[i]), end = ' ')
        print()
        
        while (1):
            out.append([])
            for i in range(n):
                out[-1].append(arr[i][indices[i]])
            next = n - 1
            while (next >= 0 and
                (indices[next] + 1 >= len(arr[next]))):
                next-=1
            if (next < 0):
                return out
            indices[next] += 1
            for i in range(next + 1, n):
                indices[i] = 0
        return out

    def computeObj(self, Y):
        Y = np.array(Y)

        y_mean = sum(Y)/len(Y)

        val = 0

        for y in Y:
            val += self.distance(y, y_mean)

        return val


        
    def computeZMvY(self, y):
        dist = collections.OrderedDict()
        for v in self.Y:
            dist[v] = self.distance(y, v)
        dist = sorted(dist.items(), key=lambda kv: kv[1])
        return dist[:self.M]

In [5]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

solver = Solver(problem_vectors, 3, 4, cos_sim)

In [53]:
subset_vec,_ = solver.solve()
for i in range(len(subset_vec)):
    subset_vec[i] = subset_vec[i][0]
subset_vec

15 objects remaining
generating ByhH. number of elements in each dimension:  28 4 11 4 
14 objects remaining


[(2, 0, 0, 0), (0, 1, 0, 0), (1, 0, 0, 0)]

In [73]:
subset_problem_id = []
for i in range(len(problem_vectors)):
    vec = tuple(problem_vectors[i])
    if vec in subset_vec:
        for id in doc_vector_ind:
            if doc_vector_ind[id]==i:
                subset_problem_id.append(id)
                break

In [74]:
subset_problem_id

['1407E', '1387A', '1388C', '1368E']

In [78]:
for i in problems:
    if i.problem_id in subset_problem_id:
        print (i.statement, end = '\n\n')

Egor is a famous Russian singer, rapper, actor and blogger, and finally he decided to give a concert in the sunny Republic of Dagestan.There are $$$n$$$ cities in the republic, some of them are connected by $$$m$$$ directed roads without any additional conditions. In other words, road system of Dagestan represents an arbitrary directed graph. Egor will arrive to the city $$$1$$$, travel to the city $$$n$$$ by roads along some path, give a concert and fly away.As any famous artist, Egor has lots of haters and too annoying fans, so he can travel only by safe roads. There are two types of the roads in Dagestan, black and white: black roads are safe at night only, and white roads — in the morning. Before the trip Egor's manager's going to make a schedule: for each city he'll specify it's color, black or white, and then if during the trip they visit some city, the only time they can leave it is determined by the city's color: night, if it's black, and morning, if it's white. After creating 